In [1]:
!pip install  -q pyngrok
!pip install  -q "unstructured[all-docs]"
!pip install  -q langchain
!pip install  -q sentence-transformers
!pip install  -q rank-bm25
!pip install  -q langchain_community
!pip install  -q langchain_huggingface
!pip install  -q sentence_transformers
!pip install  -q faiss-cpu
!pip install  -q fastapi
!pip install  -q uvicorn
!pip install  -q pydantic>=2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 80.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 99.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 9.3 

In [2]:
%%writefile BACKEND2.py

import google.generativeai
import os
import json
import re
from kaggle_secrets import UserSecretsClient
from fastapi import FastAPI, UploadFile, Form
from pydantic import BaseModel, Field
import time
from typing import List, Dict, Any
from fastapi.middleware.cors import CORSMiddleware

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")
google.generativeai.configure(api_key=secret_value_0)

system_message = """
You are a specialized legal advisory assistant operating strictly under the Indian legal framework.  
Your task is to analyze ONLY the case provided by the user and produce a structured, court-style legal advisory.  

CRITICAL: You must respond with ONLY a valid JSON object. No markdown, no explanations, no extra text.

Required JSON structure:
{
  "title": "Brief case title or Legal Advisory to [Client Name]",
  "petitioner": "Name of the petitioner",
  "respondent": "Name of the respondent",
  "successProbability": "X%" (only percentage, e.g., "75%"),
  "likelyOutcome": "Detailed prediction of probable result with legal reasoning",
  "strongPoints": "List of petitioner's strongest legal and factual arguments",
  "weakPoints": "List of petitioner's weakest or most vulnerable arguments", 
  "legalPrecedents": "List of 5 relevant Indian cases with citations and brief explanations",
  "strategies": "Possible remedies and compliance steps if petitioner loses",
  "penalties": "Potential penalties and liabilities under relevant Indian statutes",
  "finalAdvisory": "Realistic legal feedback combining all factors and probable outcome implications"
}

Analysis Guidelines:
- Identify Petitioner(s) and Respondent(s) clearly
- Predict probable case result with legal reasoning
- Assign percentage (0-100%) for petitioner's success chance
- List 5 directly relevant Indian cases with citations
- Explain strongest and weakest arguments under Indian law
- Provide comprehensive final advisory with remedies and penalties

Legal Framework:
- Prioritize Indian Constitution, statutes, and Supreme Court/High Court judgments
- Apply proportionality test, fundamental rights, and statutory compliance
- Use legally precise, court-advisory tone
- Ensure reasoning is consistent with Indian legal environment
- Include only directly relevant precedents, not generic cases

IMPORTANT: Return ONLY the JSON object. No extra text, markdown, or explanations outside the JSON.
"""

gemini = google.generativeai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=system_message
)

def extract_json_from_response(response_text: str) -> dict:
    """Extract and parse JSON from Gemini response, handling various formats"""
    try:
        # Clean the response
        cleaned = response_text.strip()
        
        # Remove markdown code blocks if present
        if '```' in cleaned:
            # Extract content between code blocks
            json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', cleaned, re.DOTALL)
            if json_match:
                cleaned = json_match.group(1)
        
        # Find JSON object in the text
        json_match = re.search(r'\{.*\}', cleaned, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            return json.loads(json_str)
        
        # If no JSON found, try parsing the entire response
        return json.loads(cleaned)
        
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print(f"Response text: {response_text}")
        # Return fallback structure
        return {
            "title": "Legal Case Analysis",
            "petitioner": "Not specified",
            "respondent": "Not specified", 
            "successProbability": "Not specified",
            "likelyOutcome": "Analysis could not be parsed. Raw response: " + response_text,
            "strongPoints": "Please check raw response",
            "weakPoints": "Please check raw response",
            "legalPrecedents": "Please check raw response",
            "strategies": "Please check raw response", 
            "penalties": "Please check raw response",
            "finalAdvisory": "JSON parsing failed. Please try again."
        }

def answer(question: str) -> dict:
    """Get structured legal analysis from Gemini"""
    try:
        user_msg = "Analyze this case and provide structured JSON response: " + question
        response = gemini.generate_content(user_msg)
        
        print(f"Raw Gemini response: {response.text}")  # Debug logging
        
        # Extract and parse JSON
        parsed_response = extract_json_from_response(response.text)
        
        # Validate required fields
        required_fields = ["title", "petitioner", "respondent", "successProbability", 
                          "likelyOutcome", "strongPoints", "weakPoints", "legalPrecedents",
                          "strategies", "penalties", "finalAdvisory"]
        
        for field in required_fields:
            if field not in parsed_response:
                parsed_response[field] = "Not provided"
        
        return parsed_response
        
    except Exception as e:
        print(f"Error in answer function: {e}")
        return {
            "title": "Error in Analysis",
            "petitioner": "Error",
            "respondent": "Error",
            "successProbability": "Error",
            "likelyOutcome": f"Error occurred: {str(e)}",
            "strongPoints": "Error in processing",
            "weakPoints": "Error in processing",
            "legalPrecedents": "Error in processing",
            "strategies": "Error in processing",
            "penalties": "Error in processing", 
            "finalAdvisory": "Please try again"
        }

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Updated response model for JSON objects
class SubmissionResponse(BaseModel):
    answers: List[Dict[str, Any]] = Field(...)  # Changed to Dict for JSON objects

class SubmissionRequest(BaseModel):
    questions: List[str] = Field(...)

@app.get("/")
def read_root():
    return {"Status of case outcome prediction": "API is running."}

@app.post("/caseoutcomeprediction", response_model=SubmissionResponse)
def caseoutcomeprediction(request: SubmissionRequest):
    print("New Request Received")
    start_time = time.time()
    answers = []
    
    for question in request.questions:
        response = answer(question)  # This now returns a dict
        answers.append(response)
        
    print("Generated answers.")
    total_time = time.time() - start_time
    print(f"Request was processed in {total_time:.2f} seconds.")
    
    return SubmissionResponse(answers=answers)

Writing BACKEND2.py


In [3]:
from pyngrok import ngrok
!ngrok config add-authtoken 32tBPN0LJscI4hVwmdpFsgRCGpD_6XU5NeYodcCerxKiy1tw5

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [4]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok
nest_asyncio.apply()
ngrok.kill()
public_url=ngrok.connect(addr="8001",proto="http",hostname="stylish-onie-slung.ngrok-free.app")
print(public_url)
uvicorn.run("BACKEND2:app", host="0.0.0.0", port=8001, reload=True)

INFO:     Will watch for changes in these directories: ['/kaggle/working']
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)
INFO:     Started reloader process [36] using StatReload


NgrokTunnel: "https://stylish-onie-slung.ngrok-free.app" -> "http://localhost:8001"


INFO:     Started server process [206]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


INFO:     103.132.31.50:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.132.31.50:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     103.132.31.50:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.132.31.50:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     103.132.31.50:0 - "OPTIONS /caseoutcomeprediction HTTP/1.1" 200 OK
New Request Received
Raw Gemini response: ```json
{
  "title": "Legal Advisory to ABC Technologies Pvt. Ltd.",
  "petitioner": "ABC Technologies Pvt. Ltd.",
  "respondent": "State of Maharashtra",
  "successProbability": "50%",
  "likelyOutcome": "The Bombay High Court is likely to acknowledge the State's legitimate interest in regulating AI models trained on sensitive public data, especially concerning privacy, misinformation, and public safety, upholding its power under Article 19(6) of the Constitution of India. However, if ABC Technologies can effectively substantiate its claims of an opaque, arbitrary, or politically influenced licensing process, the Court may strike 

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [206]
INFO:     Stopping reloader process [36]
